# Giriş işlemleri

In [ ]:
%%capture
!pip install trl bitsandbytes

## Kütüphane içe aktarma

In [ ]:
import os
import pandas as pd
import torch
from trl import SFTTrainer, SFTConfig
from datasets import Dataset
from typing import List, Optional, Callable, Any, Dict, Union, Literal, Tuple
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig, PreTrainedTokenizer)
from peft import LoraConfig, get_peft_model

In [ ]:
def is_colab() -> bool:
    """
    Google Colab ortamında çalışıp çalışmadığını kontrol eden fonksiyon.

    Args:
        None

    Returns:
        bool: Eğer kod Google Colab'da çalışıyorsa True, aksi halde False döndürür
    """
    try:
        import google.colab
        return True
    except ImportError:
        return False

In [ ]:
# Kök dizin belirleme
if is_colab():
    """
    Eğer kod Google Colab ortamında çalışıyorsa, Google Drive'ı bağlar ve
    kök dizini Google Drive içindeki "turkish_gpt2_finetuning" klasörü olarak ayarlar.
    """
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive'ı Colab ortamına bağlar
    kok_dizin = "/content/drive/MyDrive/turkish_gpt2_finetuning"  # Drive içindeki çalışma klasörünü belirler
else:
    """
    Eğer kod yerel bir ortamda çalışıyorsa, kök dizini mevcut çalışma dizini olarak ayarlar.
    """
    kok_dizin = os.getcwd()  # Mevcut çalışma dizinini alır

# Belirlenen kök dizini kullanıcıya bilgi olarak gösterir
print(f"Kök dizin: {kok_dizin}\n Not: eğer colab kullanıyorsanız, dizini değiştirmeniz gerekir.")

## Veri kümesi biçimlendirme, yol ve model yolu tanımlamaları

In [ ]:
# Veri kümesi yolu ve karıştırma seed değeri tanımlamaları
veri_kumesi_yolu = os.path.join(kok_dizin, "soru_cevap.csv")
veri_kumesi_karistirma = 571  # Karıştırmada kullanılacak sabit seed değeri
sonuc_dizini = os.path.join(kok_dizin, "sonuclar")  # Sonuçların kaydedileceği dizin

# Model kaydetme dizinleri
# gpt4o verisiyle eğitilmiş modeller
gpt2_medium_kaydetme_dizini_gpt4o = os.path.join(kok_dizin, "gpt2_medium_gpt4o")
gpt2_large_kaydetme_dizini_gpt4o = os.path.join(kok_dizin, "gpt2_large_gpt4o")

# deepseek verisiyle eğitilmiş modeller
gpt2_medium_kaydetme_dizini_deepseek = os.path.join(kok_dizin, "gpt2_medium_deepseek")
gpt2_large_kaydetme_dizini_deepseek = os.path.join(kok_dizin, "gpt2_large_deepseek")

# Model adı tanımlamaları
gpt2_medium_model_adi = "ytu-ce-cosmos/turkish-gpt2-medium"
gpt2_large_model_adi = "ytu-ce-cosmos/turkish-gpt2-large"

# Veri kümesi sütun tanımlamaları
soru_sutunu = "Soru"
gpt4o_cevap_sutunu = "gpt4o cevabı"
deepseek_cevap_sutunu = "deepseek cevabı"

# Özel token tanımlamaları (eğitim formatı için)
soru_baslangic = "<SORU>"
soru_bitis = "</SORU>"
cevap_baslangic = "<CEVAP>"
cevap_bitis = "</CEVAP>"
ornek_bitis = "<|endoftext|>"  # GPT-2'nin EOS tokeni

# Tokenizer'a eklenecek özel tokenler listesi
ozel_tokenler = [soru_baslangic, soru_bitis, cevap_baslangic, cevap_bitis]

## Veri kümesi okuma

In [ ]:
# CSV dosyasını okuma
try:
    # CSV dosyasını okuyoruz
    df = pd.read_csv(veri_kumesi_yolu)

    # İstenen sütunların varlığını kontrol ediyoruz
    gereken_sutunlar = ["Soru", gpt4o_cevap_sutunu, deepseek_cevap_sutunu]

    for sutun in gereken_sutunlar:
        if sutun not in df.columns:
            print(f"'{sutun}' sütunu veri kümesinde bulunamadı!")

    # İstenen sütunları ayıklama
    soru_cevap_df = df[gereken_sutunlar]

    # Doğrulama için ilk birkaç satırı görüntüleme
    print("Veri kümesinin ilk birkaç satırı:")
    print(soru_cevap_df.head(5))

    print(f"\nToplam {len(soru_cevap_df)} soru-cevap çifti bulundu.")

except FileNotFoundError:
    print(f"'{veri_kumesi_yolu}' dosyası bulunamadı. Lütfen dosya yolunu kontrol ediniz.")
except Exception as e:
    print(f"Veri okuma hatası: {e}")

In [ ]:
# Veri kümesini belirtilen seed değeriyle karıştırma
shuffled_df = soru_cevap_df.sample(frac=1, random_state=veri_kumesi_karistirma)

# İndeksleri sıfırlama
shuffled_df = shuffled_df.reset_index(drop=True)

# Karıştırılmış veri kümesinin ilk birkaç satırını gösterme
print("Karıştırılmış veri kümesinin ilk birkaç satırı:")
print(shuffled_df.head(5))

print(f"\nKarıştırılmış veri kümesi boyutu: {len(shuffled_df)} satır")

# Veri kümesi işleme

## Veri kümesi biçimlendirme fonksiyonu

In [ ]:
def veri_kumesini_egitim_formatina_donustur(df: pd.DataFrame, cevap_sutunu: str,
                                            soru_baslangic: str = soru_baslangic,
                                            soru_bitis: str = soru_bitis,
                                            cevap_baslangic: str = cevap_baslangic,
                                            cevap_bitis: str = cevap_bitis,
                                            ornek_bitis: str = ornek_bitis) -> list:
    """
    Veri kümesindeki soru ve cevapları GPT-2 eğitimi için uygun formata dönüştürür.
    Metinsel formatta örnekler döndürür, tokenize işlemi uygulamaz.

    Args:
        df: Soru ve cevapları içeren DataFrame
        cevap_sutunu: Cevap metinlerini içeren sütunun adı
        soru_baslangic: Soru başlangıç etiketi
        soru_bitis: Soru bitiş etiketi
        cevap_baslangic: Cevap başlangıç etiketi
        cevap_bitis: Cevap bitiş etiketi
        ornek_bitis: Her örneğin sonunu belirten etiket

    Returns:
        list: Eğitim için hazırlanmış metinsel örnekler listesi
    """

    egitim_metinleri = []

    # Her bir soru-cevap çifti için formatlı metinler oluştur
    for _, satir in df.iterrows():
        soru = satir["Soru"].strip()
        cevap = satir[cevap_sutunu].strip()

        # Soru ve cevabı belirli bir formatta birleştir
        bicimlendirilmis_metin = f"{soru_baslangic} {soru} {soru_bitis} {cevap_baslangic} {cevap} {cevap_bitis}{ornek_bitis}"

        egitim_metinleri.append(bicimlendirilmis_metin)

    print(f"Toplam {len(egitim_metinleri)} adet eğitim örneği oluşturuldu.")
    print(f"Örnek biçimi: {soru_baslangic} [Soru] {soru_bitis} {cevap_baslangic} [Cevap] {cevap_bitis}{ornek_bitis}")
    return egitim_metinleri

## Veri kümelerini işle ve oluştur

In [ ]:
veri_kumesi_gpt4o = veri_kumesini_egitim_formatina_donustur(shuffled_df, gpt4o_cevap_sutunu)
veri_kumesi_deepseek = veri_kumesini_egitim_formatina_donustur(shuffled_df, deepseek_cevap_sutunu)

# Eğitim işlemleri

## Model Yükleme ve Eğitme Fonksiyonları

### Eğitim argümanları oluşturma fonksiyonu

In [ ]:
import math
max_seq_length = 1024   # üstte sabit tanım

def training_arguments_getir(
    kaydetme_dizin      : str,
    veri_kumesi_uzunlugu: int,          # len(dataset)
    *,
    # hiper-parametreler
    learning_rate : float = 1e-4,
    batch_size    : int   = 2,
    grad_accum_steps    : int   = 32,
    # eğitim süresi
    epochs        : int   = 1,
    # sekuans uzunluğu
    max_seq_length: int   = max_seq_length,
    # “oran”lar
    save_per_epoch: int   = 4,          # her epoch’ta 4 checkpoint
    log_per_epoch : int   = 12,          # her epoch’ta 8 loss logu
) -> SFTConfig:

    # --------------------- türetilen değerler -----------------------------
    ngpu = max(torch.cuda.device_count(), 1)
    effective_batch = batch_size * grad_accum * ngpu

    steps_per_epoch = math.ceil(veri_kumesi_uzunlugu / effective_batch)
    max_steps       = epochs * steps_per_epoch
    save_steps      = max(1, steps_per_epoch // save_per_epoch)
    logging_steps   = max(1, steps_per_epoch // log_per_epoch)

    print(f"[INFO] veri: {veri_kumesi_uzunlugu} örnek  | "
          f"gpu: {ngpu}  |  effective_batch: {effective_batch}  |  "
          f"steps/epoch: {steps_per_epoch}  |  total steps: {max_steps}")

    # --------------------- ortak argümanlar --------------------------------
    cfg_kwargs = dict(
        output_dir = kaydetme_dizin,

        # hiper-parametreler
        learning_rate               = learning_rate,
        per_device_train_batch_size = batch_size,
        gradient_accumulation_steps = grad_accum_steps,

        # sekuans uzunluğu
        max_seq_length = max_seq_length,

        # optimizasyon / planlayıcı
        optim             = "paged_adamw_8bit",
        weight_decay      = 0.1,
        warmup_ratio      = 0.1,
        lr_scheduler_type = "cosine",
        max_grad_norm     = 0.1,

        # loglama
        logging_dir      = os.path.join(kaydetme_dizin, "logs"),
        logging_strategy = "steps",
        logging_steps    = logging_steps,
        report_to        = "tensorboard",
        run_name         = os.path.basename(kaydetme_dizin),

        # checkpoint
        save_strategy = "steps",
        save_steps    = save_steps,

        # epoch tabanlı
        num_train_epochs = epochs,
        max_steps        = -1,          # Trainer epoch modunda -1 ister
    )

    return SFTConfig(**cfg_kwargs)

### Model Yükleme fonksiyonu

In [ ]:
def model_ve_tokenizer_yukle(
    model_adi: str,
    max_seq_length: int = max_seq_length,
    lora_rank: int = 16,
    random_state: int = veri_kumesi_karistirma,
    target_modules: List[str] = ("c_attn", "c_proj", "c_fc"),
    ozel_tokenler: List[str] = None,
    quant_mode: Literal["4bit", "8bit", "16bit"] = "16bit",
) -> Tuple[torch.nn.Module, PreTrainedTokenizer]:
    """
    • quant_mode = "4bit"  -> 4-bit QLoRA
    • quant_mode = "8bit"  -> 8-bit (bnb)
    • quant_mode = "16bit" -> fp16 / bf16, quantization yok
    """
    # ----------------------- tokenizer -------------------------------------
    tokenizer = AutoTokenizer.from_pretrained(model_adi, use_fast=True)
    tokenizer.padding_side, tokenizer.truncation_side = "right", "right"
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})

    # ----------------------- model (quant / dtype) -------------------------
    quant_cfg = None
    model_kwargs = dict(device_map="auto", trust_remote_code=True)

    if quant_mode == "4bit":
        quant_cfg = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=
                torch.bfloat16 if torch.cuda.get_device_capability(0)[0] >= 8
                else torch.float16,
        )
        model_kwargs["quantization_config"] = quant_cfg

    elif quant_mode == "8bit":
        quant_cfg = BitsAndBytesConfig(load_in_8bit=True)
        model_kwargs["quantization_config"] = quant_cfg

    else:  # "16bit"
        # fp16 ile yükle; Ampere (8.x) ve üstü kartlarda bf16 da seçilebilir
        model_kwargs["torch_dtype"] = (
            torch.bfloat16 if torch.cuda.get_device_capability(0)[0] >= 8
            else torch.float16
        )

    model = AutoModelForCausalLM.from_pretrained(model_adi, **model_kwargs)
    model.config.max_position_embeddings = max_seq_length
    model.config.use_cache = False
    model.gradient_checkpointing_enable()

    # ----------------------- LoRA -----------------------------------------
    lora_cfg = LoraConfig(
        r=lora_rank,
        lora_alpha=lora_rank,
        target_modules=list(target_modules),
        bias="none",
        task_type="CAUSAL_LM",
        fan_in_fan_out=True,
    )
    model = get_peft_model(model, lora_cfg)
    model.print_trainable_parameters()

    # ----------------------- özel tokenlar ---------------------------------
    if ozel_tokenler:
        tokenizer.add_special_tokens({"additional_special_tokens": ozel_tokenler})
        model.resize_token_embeddings(len(tokenizer))

    torch.cuda.manual_seed(random_state)
    print(f"{model_adi} '{quant_mode}' modunda yüklendi (+LoRA).")
    return model, tokenizer

### Eğitici Getirme fonksiyonu

In [ ]:
def trainer_getir(model, training_args, veri_kumesi, tokenizer):
    toks = tokenizer(
        veri_kumesi,
        padding="max_length",
        truncation=True,
        max_length=training_args.max_seq_length,
    )

    # torch tensörleri listeye çeviriyoruz
    data_dict = {
        "input_ids":      [ids for ids in toks["input_ids"]],
        "attention_mask": [mask for mask in toks["attention_mask"]],
    }
    train_dataset = Dataset.from_dict(data_dict)
    train_dataset.set_format(type="torch")

    print(f"Eğitim için {len(train_dataset)} örnek hazırlandı.")

    trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
    )
    return trainer


### Veri kümesi özelinde model eğitme fonksiyonu

In [ ]:
import json
def model_egit_ve_kaydet(
    model,
    tokenizer,
    veri_kumesi,
    model_kaydetme_dizini,
):
    """
    Eğitim döngüsünü başlatır, tamamlandığında modeli ve tokenizer’ı kaydeder.
    """
    os.makedirs(model_kaydetme_dizini, exist_ok=True)

    training_args = training_arguments_getir(
        kaydetme_dizin=model_kaydetme_dizini,
        max_steps=250,
        save_steps=50,
        logging_steps=25
    )

    trainer = trainer_getir(
        model         = model,
        training_args = training_args,
        veri_kumesi   = veri_kumesi,
        tokenizer     = tokenizer,
    )


    print(f"\n[Eğitim] {model_kaydetme_dizini} dizininde eğitim başlatılıyor...")
    trainer.train()
    trainer.save_model(model_kaydetme_dizini)
    tokenizer.save_pretrained(model_kaydetme_dizini)
    log_json_path = os.path.join(model_kaydetme_dizini, "training_log.json")
    with open(log_json_path, "w", encoding="utf-8") as f:
        json.dump(trainer.state.log_history, f, ensure_ascii=False, indent=4)
    print(f"Model ve tokenizer '{model_kaydetme_dizini}' dizinine kaydedildi.")

    # Belleği temizle
    del model
    torch.cuda.empty_cache()

## GPT-2 Medium modelini GPT-4o veri kümesi ile eğit

In [ ]:
print("\nGPT-2 Medium modeli GPT-4o veri kümesi ile eğitiliyor...")
model_medium, tokenizer_medium = model_ve_tokenizer_yukle(gpt2_medium_model_adi)
# GPT-4o veri kümesiyle Medium modeli eğit
model_egit_ve_kaydet(
    model_medium,
    tokenizer_medium,
    veri_kumesi_gpt4o,
    gpt2_medium_kaydetme_dizini_gpt4o,
)

## GPT-2 Medium modelini DeepSeek veri kümesi ile eğit

In [ ]:
# DeepSeek veri kümesiyle Medium modeli eğit
print("\nGPT-2 Medium modeli DeepSeek veri kümesi ile eğitiliyor...")
model_medium, tokenizer_medium = model_ve_tokenizer_yukle(gpt2_medium_model_adi)
model_egit_ve_kaydet(
    model_medium,
    tokenizer_medium,
    veri_kumesi_deepseek,
    gpt2_medium_kaydetme_dizini_deepseek,
)

## GPT-2 Large modelini GPT-4o veri kümesi ile eğit

In [ ]:
# GPT-4o veri kümesiyle Large modeli eğit
print("\nGPT-2 Large modeli GPT-4o veri kümesi ile eğitiliyor...")
model_large, tokenizer_large = model_ve_tokenizer_yukle(gpt2_large_model_adi)
model_egit_ve_kaydet(
    model_large,
    tokenizer_large,
    veri_kumesi_gpt4o,
    gpt2_large_kaydetme_dizini_gpt4o,
)

## GPT-2 Large modelini DeepSeek veri kümesi ile eğit

In [ ]:
# DeepSeek veri kümesiyle Large modeli eğit
print("\nGPT-2 Large modeli DeepSeek veri kümesi ile eğitiliyor...")
model_large, tokenizer_large = model_ve_tokenizer_yukle(gpt2_large_model_adi)
model_egit_ve_kaydet(
    model_large,
    tokenizer_large,
    veri_kumesi_deepseek,
    gpt2_large_kaydetme_dizini_deepseek,
)